# Введение в обработку естественного языка
## Урок 11. Модель Transformer-1
### 1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для тогоже корпуса что вы выбрали из предыдущего дз)

In [1]:
!pip install transformers
!pip install sacremoses
!pip install torch torchvision
!pip install corus
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 10.0 MB/s 
     |████████████████████████████████| 6.6 MB 33.1 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 6.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d09b64ec9c93f353f87e11df02ea6db7ef87036e93e526f034926334af070b5a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.

In [2]:
!wget http://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2022-07-12 15:10:34--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.13M  19.1MB/s    in 0.7s    

2022-07-12 15:10:35 (19.1 MB/s) - ‘rus-eng.zip’ saved [14819554/14819554]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [3]:
import io
import re

import pandas as pd
import numpy as np

from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from sentence_transformers import SentenceTransformer, util

import corus
from corus import load_ods_gazeta

In [6]:
def preprocess_sentence(w):
  w = w.lower().strip()

  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  w = '<start> ' + w + ' <end>'
  return w

In [7]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [8]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [9]:
def load_dataset(path, num_examples=None):
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [10]:
en, rus = create_dataset('/content/rus.txt', None)

In [11]:
df = pd.DataFrame(en, columns=['en'])
df['rus'] = rus
df.head()

,en,rus
0,<start> go . <end>,<start> марш ! <end>
1,<start> go . <end>,<start> иди . <end>
2,<start> go . <end>,<start> идите . <end>
3,<start> hi . <end>,<start> здравствуйте . <end>
4,<start> hi . <end>,<start> привет ! <end>


In [12]:
def remove_start_stop(tp):
  out = []
  for i in range(len(tp)-1):
    out.append(tp[i].replace('<start>', '').replace('<end>', ''))
  return out

In [20]:
en, rus = remove_start_stop(en), remove_start_stop(rus)
del df
df = pd.DataFrame(en, columns=['en'])
df['rus'] = rus
df.head()

,en,rus
0,go .,марш !
1,go .,иди .
2,go .,идите .
3,hi .,здравствуйте .
4,hi .,привет !


In [14]:
mname = "facebook/wmt19-en-ru"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

In [15]:
def translate(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(input_ids)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [22]:
part = df.loc[:20,:]
part['translation'] = part['en'].apply(np.vectorize(translate))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
part.head(20)

,en,rus,translation
0,go .,марш !,go.
1,go .,иди .,go.
2,go .,идите .,go.
3,hi .,здравствуйте .,hi.
4,hi .,привет !,hi.
5,hi .,хай .,hi.
6,hi .,здрасте .,hi.
7,hi .,здоро во !,hi.
8,hi .,приветик !,hi.
9,run !,беги !,run!


### 2. скачиваем готовый новостной датасет
!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

        ```
        # пример работы с ним 
        from corus import load_ods_gazeta
        path = 'gazeta.csv.gz'
        records = load_ods_gazeta(path)
        next(records)
        ```

    реализовать метод поиска ближайших статей
    (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу)
    визуально оценить качество

In [4]:
!wget -O gazeta.csv.gz -qq --no-check-certificate https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

In [5]:
path = '/content/gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

NewsRecord(
    timestamp=datetime.datetime(2008, 11, 21, 15, 19, 14),
    url='https://www.gazeta.ru/news/business/2008/11/21/n_1298950.shtml',
    edition=None,
    topics='Бизнес',
    authors=None,
    title='Госдума сокращает срок действия ставки экспортных пошлин на нефть',
    text='Госдума приняла сегодня в первом чтении и сразу в целом поправки в закон «О таможенном тарифе», сокращающие срок действия ставки экспортных пошлин на нефть с 2-х до 1-го месяца.nnДля установления средних цен на нефть марки Urals и расчета экспортных пошлин правительство России в течение двух месяцев проводит мониторинг на международных рынках нефтяного сырья (средиземноморском и роттердамском), экспортные пошлины на нефть устанавливаются также раз в два месяца.nnСокращение на месяц периода мониторинга (с 15-го числа каждого календарного месяца по 14-е число следующего месяца) и соответственно срока действия ставок экспортных пошлин «позволит более оперативно реагировать на изменения экономической сит

In [6]:
df = pd.DataFrame(records)

In [7]:
model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/356 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [8]:
sentences = df[6][:100]

In [9]:
embeddings = model.encode(sentences, convert_to_tensor=True)

In [10]:
def get_similar(embeddings, sentence, n):
    
    assert n > 0, "Bad n"
    s_emb =  model.encode([sentence], convert_to_tensor=True)

    cosine_scores = util.cos_sim(s_emb, embeddings)

    max_scores_idx = np.argpartition(cosine_scores[0], -n)[-n:]
    
    return cosine_scores, max_scores_idx

In [11]:
rnd = np.random.randint(100, df.shape[0])
sentence = df.iloc[rnd, 6]

sentence

cosine_scores, max_scores_idx = get_similar(embeddings, sentence, 3)

for t in sentences[max_scores_idx.tolist()]:
    print(t)
    print("\n")

Белорусский оппозиционер Сергей Коваленко, которого правозащитники относят к политзаключенным, освобожден из могилевской колонии, сообщаетn«Интерфакс»nсо ссылкой на местные белорусские СМИ.nВ настоящее время Коваленко с женой Еленой находится у своей матери.nВ январе 2010 года Коваленко вывесил на новогодней елке в Витебске бело-красно-белый флаг. После этого он был обвинен по части 1 статьи 339 белорусского УК (умышленные действия, грубо нарушающие общественный порядок) и части 2 статьи 363 (сопротивление работнику правоохранительных органов при исполнении им служебных обязанностей). nnВ декабре 2011 года оппозиционер был задержан и обвинен в «уклонении от отбытия наказания» и приговорен к двум годам лишения свободы. После задержания Коваленко объявил голодовку, которая длилась около двух месяцев.nВ июне этого года стало известно, что Коваленко написал на имя Александра Лукашенко прошение о помиловании. В минувшее воскресенье президент, общаясь с журналистами, подтвердил, что на его и

In [12]:
rnd = np.random.randint(100, df.shape[0])
sentence = df.iloc[rnd, 6]

sentence

cosine_scores, max_scores_idx = get_similar(embeddings, sentence, 3)

for t in sentences[max_scores_idx.tolist()]:
    print(t)
    print("\n")

Белорусский оппозиционер Сергей Коваленко, которого правозащитники относят к политзаключенным, освобожден из могилевской колонии, сообщаетn«Интерфакс»nсо ссылкой на местные белорусские СМИ.nВ настоящее время Коваленко с женой Еленой находится у своей матери.nВ январе 2010 года Коваленко вывесил на новогодней елке в Витебске бело-красно-белый флаг. После этого он был обвинен по части 1 статьи 339 белорусского УК (умышленные действия, грубо нарушающие общественный порядок) и части 2 статьи 363 (сопротивление работнику правоохранительных органов при исполнении им служебных обязанностей). nnВ декабре 2011 года оппозиционер был задержан и обвинен в «уклонении от отбытия наказания» и приговорен к двум годам лишения свободы. После задержания Коваленко объявил голодовку, которая длилась около двух месяцев.nВ июне этого года стало известно, что Коваленко написал на имя Александра Лукашенко прошение о помиловании. В минувшее воскресенье президент, общаясь с журналистами, подтвердил, что на его и